# Import Libraries

In [8]:
from pipeline import RawDataPipeline
import missingno as msno
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import os

from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler, MinMaxScaler, OrdinalEncoder
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV


from utils import get_district, swimming_pool, fitness, balcony

pd.options.display.float_format = '{:.2f}'.format

# Check Working Directory

In [9]:
os.getcwd()

'/Users/samuelwong/Projects/PropertyGuru Analysis/PropertyGuruApp'

# Import Raw Data

In [11]:
raw_pipeline = RawDataPipeline()
df = raw_pipeline.run()

/Users/samuelwong/Projects/PropertyGuru Analysis/PropertyGuruApp


In [12]:
df_ori = df.copy()

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2667 entries, 1 to 2667
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   price                2661 non-null   object 
 1   listing_title        2664 non-null   object 
 2   name                 2661 non-null   object 
 3   sqft                 2643 non-null   object 
 4   bedrooms             2630 non-null   object 
 5   bathrooms            2626 non-null   float64
 6   address              2661 non-null   object 
 7   price_per_sqft       2616 non-null   object 
 8   listing_tags         2666 non-null   object 
 9   listings_desc        2664 non-null   object 
 10  property_title_type  1546 non-null   object 
 11  furnishing           1974 non-null   object 
 12  listed_on            2664 non-null   object 
 13  tenure               2339 non-null   object 
 14  property_type        2664 non-null   object 
 15  psf_det              2585 non-null   o

In [14]:
df.head()

,price,listing_title,name,sqft,bedrooms,bathrooms,address,price_per_sqft,listing_tags,listings_desc,property_title_type,furnishing,listed_on,tenure,property_type,psf_det,floor_size_det,facilities,url
id,,,,,,,,,,,,,,,,,,,
1,"1,288,000",ECO SANCTUARY,ECO SANCTUARY,2622,5,5.00,"Lot 41296 Persiaran Eco Sanctuary, Telok Pangl...",491.23,2-storey Terraced House,\nDescription\nTerraza 2-Storey Superlink Hous...,NaN,Partially Furnished,19 hours ago,Leasehold,2-storey Terraced House For Sale,RM 491.23 psf,2622 sqft,"Air-Conditioning, Balcony, Water Heater, Baske...",https://www.propertyguru.com.my/property-listi...
2,"1,600,000",Kinrara Residence - 3 Storey Link House,Kinrara Residence - 3 Storey Link House,1847,5,5.00,"Jalan DU 4/7, Puchong, Selangor",866.27,3-storey Terraced House,\nDescription\n3-storey Terraced House For Sal...,NaN,Unfurnished,5 hours ago,Leasehold,3-storey Terraced House For Sale,RM 866.27 psf,1847 sqft,"Air-Conditioning, Balcony, Water Heater, Car P...",https://www.propertyguru.com.my/property-listi...
3,"400,000",Menara Mutiara,Menara Mutiara,1100,3,2.00,"Jalan 11 Off Jalan 1, Taman TAR, Ampang Jaya, ...",363.64,Condominium,"\nDescription\nLush greenery, mountain near KL...",Strata,Partially Furnished,2 days ago,Leasehold,Condominium For Sale,RM 363.64 psf,1100 sqft,"Air-Conditioning, Balcony, Bathtub, Maidsroom,...",https://www.propertyguru.com.my/property-listi...
4,"665,000","Ellis, Bandar Bukit Raja","Ellis, Bandar Bukit Raja",1535,4,3.00,"Jalan Sumazau 3/KU5, Bandar Bukit Raja, Klang,...",433.22,2-storey Terraced House,\nDescription\nElsa bandar bukit raja\n ...,Individual,Unfurnished,1 day ago,Freehold,2-storey Terraced House For Sale,RM 433.22 psf,1535 sqft,"Car Park, Perimeter Fencing, Playground",https://www.propertyguru.com.my/property-listi...
5,"599,000",Seri Bangi Seksyen 8 Bandar Baru Bangi,Seri Bangi Seksyen 8 Bandar Baru Bangi,1900,4,3.00,"Jalan 8/35, Bandar Baru Bangi, Bangi, Selangor",315.26,2-storey Terraced House,"\nDescription\nRumah luas, kawasan strategik d...",Individual,Unfurnished,1 day ago,Leasehold,2-storey Terraced House For Sale,RM 315.26 psf,1900 sqft,"Air-Conditioning, Renovated, Barbeque Area, Co...",https://www.propertyguru.com.my/property-listi...


# Data Preprocessing & Cleaning

## Data types processing & Remove unnecessary features

In [15]:
df.loc[df.listing_title != df.name]

,price,listing_title,name,sqft,bedrooms,bathrooms,address,price_per_sqft,listing_tags,listings_desc,property_title_type,furnishing,listed_on,tenure,property_type,psf_det,floor_size_det,facilities,url
id,,,,,,,,,,,,,,,,,,,
879,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"1-storey Terraced House, Freehold",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.propertyguru.com.my/property-listi...
1667,NaN,The Waterlily at Wetlands Estates,NaN,NaN,NaN,NaN,NaN,NaN,"New Project, Bungalow House",\nAbout The Waterlily at Wetlands Estates\n ...,NaN,NaN,11 hours ago,Leasehold,Bungalow House For Sale,NaN,NaN,"Clubhouse, Community Garden, Perimeter Fencing...",https://www.propertyguru.com.my/property-listi...
1967,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Apartment, Leasehold, Completion: 2007",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.propertyguru.com.my/property-listi...
1994,NaN,The Waterlily at Wetlands Estates,NaN,NaN,NaN,NaN,NaN,NaN,"New Project, Bungalow House",\nAbout The Waterlily at Wetlands Estates\n ...,NaN,NaN,12 hours ago,Leasehold,Bungalow House For Sale,NaN,NaN,"Clubhouse, Community Garden, Perimeter Fencing...",https://www.propertyguru.com.my/property-listi...
2112,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.propertyguru.com.my/property-listi...
2527,NaN,Valeria Garden Homes,NaN,NaN,NaN,NaN,NaN,NaN,"New Project, 2-storey Terraced House, 2.5-stor...",\nAbout Valeria Garden Homes\n ...,NaN,NaN,1 week ago,Leasehold,2-storey Terraced House For Sale,NaN,NaN,"Children's Playground, Clubhouse, Main entranc...",https://www.propertyguru.com.my/property-listi...


1. We can see that the columns listing_title and name are having the same values except for the 10 rows shown above.
2. This is because the PropertyGuru website uses the same name for both the listing's title and listing's name.
3. Therefore, we can remove 1 of the columns, which is the 'name' columns as it contains more missing values

In [16]:
try:
    df = df.drop(columns = 'name')
except:
    pass
assert 'name' not in df.columns

In [17]:
df

,price,listing_title,sqft,bedrooms,bathrooms,address,price_per_sqft,listing_tags,listings_desc,property_title_type,furnishing,listed_on,tenure,property_type,psf_det,floor_size_det,facilities,url
id,,,,,,,,,,,,,,,,,,
1,"1,288,000",ECO SANCTUARY,2622,5,5.00,"Lot 41296 Persiaran Eco Sanctuary, Telok Pangl...",491.23,2-storey Terraced House,\nDescription\nTerraza 2-Storey Superlink Hous...,NaN,Partially Furnished,19 hours ago,Leasehold,2-storey Terraced House For Sale,RM 491.23 psf,2622 sqft,"Air-Conditioning, Balcony, Water Heater, Baske...",https://www.propertyguru.com.my/property-listi...
2,"1,600,000",Kinrara Residence - 3 Storey Link House,1847,5,5.00,"Jalan DU 4/7, Puchong, Selangor",866.27,3-storey Terraced House,\nDescription\n3-storey Terraced House For Sal...,NaN,Unfurnished,5 hours ago,Leasehold,3-storey Terraced House For Sale,RM 866.27 psf,1847 sqft,"Air-Conditioning, Balcony, Water Heater, Car P...",https://www.propertyguru.com.my/property-listi...
3,"400,000",Menara Mutiara,1100,3,2.00,"Jalan 11 Off Jalan 1, Taman TAR, Ampang Jaya, ...",363.64,Condominium,"\nDescription\nLush greenery, mountain near KL...",Strata,Partially Furnished,2 days ago,Leasehold,Condominium For Sale,RM 363.64 psf,1100 sqft,"Air-Conditioning, Balcony, Bathtub, Maidsroom,...",https://www.propertyguru.com.my/property-listi...
4,"665,000","Ellis, Bandar Bukit Raja",1535,4,3.00,"Jalan Sumazau 3/KU5, Bandar Bukit Raja, Klang,...",433.22,2-storey Terraced House,\nDescription\nElsa bandar bukit raja\n ...,Individual,Unfurnished,1 day ago,Freehold,2-storey Terraced House For Sale,RM 433.22 psf,1535 sqft,"Car Park, Perimeter Fencing, Playground",https://www.propertyguru.com.my/property-listi...
5,"599,000",Seri Bangi Seksyen 8 Bandar Baru Bangi,1900,4,3.00,"Jalan 8/35, Bandar Baru Bangi, Bangi, Selangor",315.26,2-storey Terraced House,"\nDescription\nRumah luas, kawasan strategik d...",Individual,Unfurnished,1 day ago,Leasehold,2-storey Terraced House For Sale,RM 315.26 psf,1900 sqft,"Air-Conditioning, Renovated, Barbeque Area, Co...",https://www.propertyguru.com.my/property-listi...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2663,"1,250,000",Xania Puchong,3800.00,5,5.00,"Cyberjaya, Selangor",328.95,Terraced House,\nDescription\nXania Puchong\n ...,Individual,Unfurnished,3 days ago,Leasehold,Terraced House For Sale,RM 328.95 psf,3800 sqft,"Adult Fitness Stations, Badminton hall, Barbeq...",https://www.propertyguru.com.my/property-listi...
2664,"980,000",Tenderfields @ Eco Majestic,2750.00,5,5.00,"No. 1, Lingkaran Eco Majestic, Eco Majestic, S...",356.36,Cluster House,"\nDescription\nSpecialist at Eco Majestic, Man...",Strata,NaN,1 day ago,Freehold,Cluster House For Sale,RM 356.36 psf,2750 sqft,"Adult Fitness Stations, Badminton hall, Barbeq...",https://www.propertyguru.com.my/property-listi...
2665,"880,000",Lumi Tropicana,1084.00,3,2.00,"Lumi Tropicana, Persiaran Tropicana, Tropicana...",811.81,Service Residence,\nDescription\nPartially Furnished corner with...,Individual,Partially Furnished,3 days ago,Leasehold,Service Residence For Sale,RM 811.81 psf,1084 sqft,"Adult Fitness Stations, Badminton hall, Barbeq...",https://www.propertyguru.com.my/property-listi...


In [18]:
# we can format the price column to float type

df['price'] = (
    df['price']
    .astype('str')
    .str.replace(',', '')
    .astype('float')
)

assert df['price'].dtype == 'float'

In [19]:
df.head()

,price,listing_title,sqft,bedrooms,bathrooms,address,price_per_sqft,listing_tags,listings_desc,property_title_type,furnishing,listed_on,tenure,property_type,psf_det,floor_size_det,facilities,url
id,,,,,,,,,,,,,,,,,,
1,1288000.00,ECO SANCTUARY,2622,5,5.00,"Lot 41296 Persiaran Eco Sanctuary, Telok Pangl...",491.23,2-storey Terraced House,\nDescription\nTerraza 2-Storey Superlink Hous...,NaN,Partially Furnished,19 hours ago,Leasehold,2-storey Terraced House For Sale,RM 491.23 psf,2622 sqft,"Air-Conditioning, Balcony, Water Heater, Baske...",https://www.propertyguru.com.my/property-listi...
2,1600000.00,Kinrara Residence - 3 Storey Link House,1847,5,5.00,"Jalan DU 4/7, Puchong, Selangor",866.27,3-storey Terraced House,\nDescription\n3-storey Terraced House For Sal...,NaN,Unfurnished,5 hours ago,Leasehold,3-storey Terraced House For Sale,RM 866.27 psf,1847 sqft,"Air-Conditioning, Balcony, Water Heater, Car P...",https://www.propertyguru.com.my/property-listi...
3,400000.00,Menara Mutiara,1100,3,2.00,"Jalan 11 Off Jalan 1, Taman TAR, Ampang Jaya, ...",363.64,Condominium,"\nDescription\nLush greenery, mountain near KL...",Strata,Partially Furnished,2 days ago,Leasehold,Condominium For Sale,RM 363.64 psf,1100 sqft,"Air-Conditioning, Balcony, Bathtub, Maidsroom,...",https://www.propertyguru.com.my/property-listi...
4,665000.00,"Ellis, Bandar Bukit Raja",1535,4,3.00,"Jalan Sumazau 3/KU5, Bandar Bukit Raja, Klang,...",433.22,2-storey Terraced House,\nDescription\nElsa bandar bukit raja\n ...,Individual,Unfurnished,1 day ago,Freehold,2-storey Terraced House For Sale,RM 433.22 psf,1535 sqft,"Car Park, Perimeter Fencing, Playground",https://www.propertyguru.com.my/property-listi...
5,599000.00,Seri Bangi Seksyen 8 Bandar Baru Bangi,1900,4,3.00,"Jalan 8/35, Bandar Baru Bangi, Bangi, Selangor",315.26,2-storey Terraced House,"\nDescription\nRumah luas, kawasan strategik d...",Individual,Unfurnished,1 day ago,Leasehold,2-storey Terraced House For Sale,RM 315.26 psf,1900 sqft,"Air-Conditioning, Renovated, Barbeque Area, Co...",https://www.propertyguru.com.my/property-listi...


In [20]:
# we can format the sqft column to float type
df['sqft'] = (
    df['sqft']
    .astype('str')
    .str.replace(',', '')
    .str.replace('acre', '')
    .str.strip()
    .astype('float')
)

assert df['sqft'].dtype == 'float'

In [21]:
# format bedrooms into float
df['bedrooms'].unique()


array(['5', '3', '4', '2', '1', '6', nan, '7', '8', 'Studio', '9', '10'],
      dtype=object)

We can see that there is a category known as 'Studio', we can encode it as 0 since studio unit means no bedroom.

In [22]:
# Define dictonary with values to map
map_dict = {'1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, '10': 10, 'Studio': 0,
                'studio': 0}

df['bedrooms'] = df['bedrooms'].astype('str').map(map_dict)

df['bedrooms'] = df['bedrooms'].astype('float')
    
assert df['bedrooms'].dtype == 'float'

In [23]:
# format bathrooms into float

df['bathrooms'].unique()

assert ('Studio' not in df['bathrooms'].unique()) and ('studio' not in df['bathrooms'].unique())

/var/folders/ss/52sqv29j2zdgspf8_wh7wppm0000gn/T/ipykernel_67212/3541889416.py:5: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  assert ('Studio' not in df['bathrooms'].unique()) and ('studio' not in df['bathrooms'].unique())


In [24]:
df['bathrooms'] = df['bathrooms'].astype('float')
    
assert df['bathrooms'].dtype == 'float'

In [25]:
# convert price per sqft into float
try:
    df['price_per_sqft'] = df['price_per_sqft'].astype('float')
except:
    # means there is string in the values
    df['price_per_sqft'] = df['price_per_sqft'].astype('str').apply(lambda x: x.replace(',', ''))
    df['price_per_sqft'] = df['price_per_sqft'].astype('float')

assert df['price_per_sqft'].dtype == 'float'

In [26]:
# we can extract the district of the property from address column
# the second last name of the address should be the district, as the last name is the state
df['address'].head()

id
1    Lot 41296 Persiaran Eco Sanctuary, Telok Pangl...
2                      Jalan DU 4/7, Puchong, Selangor
3    Jalan 11 Off Jalan 1, Taman TAR, Ampang Jaya, ...
4    Jalan Sumazau 3/KU5, Bandar Bukit Raja, Klang,...
5       Jalan 8/35, Bandar Baru Bangi, Bangi, Selangor
Name: address, dtype: object

In [27]:
# check whether all the last name of address is state
df['state'] = df['address'].astype('str') \
    .apply(lambda x: x.split(',')[-1])

In [28]:
df.state.unique()

array([' Selangor', 'nan'], dtype=object)

Ensure all the state of the listings are Selangor as the project is to analyse Selangor's house price.

In [29]:
df['district'] = df['address'].astype('str')\
                        .apply(get_district)

In [30]:
df['district'].unique()

array(['Kuala Langat', 'Puchong', 'Ampang', 'Klang', 'Bangi', 'Sepang',
       'Rawang', 'Damansara', 'Kajang', 'Cheras', 'Subang Jaya',
       'Petaling Jaya', 'Seri Kembangan', 'Cyberjaya', 'Kuala Selangor',
       'Shah Alam', 'Nilai', 'Semenyih', 'Sungai Buloh', 'Gombak',
       'Subang', 'Putrajaya', 'Hulu Langat', 'Hulu Kelang',
       'Hulu Selangor', 'Beranang', 'Ulu Klang', 'nan', 'Sabak Bernam'],
      dtype=object)

We can see that we have successfully extract the district out, but we explore the district out in the subsequent analysis.

In [31]:
# we can drop listings description as no valuable info can be obtained from it
try:
    df = df.drop(columns = 'listings_desc')
except:
    pass

assert 'listings_desc' not in df.columns

In [32]:
try:
    df = df.drop(columns = 'listed_on')
except:
    pass

assert 'listed_on' not in df.columns

It seems like the value in property_type column is similar to listing_tags, maybe can drop 1 of the columns

In [33]:
df['listing_tags']

id
1       2-storey Terraced House
2       3-storey Terraced House
3                   Condominium
4       2-storey Terraced House
5       2-storey Terraced House
                 ...           
2663             Terraced House
2664              Cluster House
2665          Service Residence
2666        Semi-Detached House
2667          Service Residence
Name: listing_tags, Length: 2667, dtype: object

In [34]:
property_type_check = df['property_type']\
                                .str.replace('For Sale','')\
                                .str.strip()

In [35]:
df.loc[df.listing_tags != property_type_check][['listing_tags']]

,listing_tags
id,
879,"1-storey Terraced House, Freehold"
1667,"New Project, Bungalow House"
1967,"Apartment, Leasehold, Completion: 2007"
1994,"New Project, Bungalow House"
2112,NaN
2527,"New Project, 2-storey Terraced House, 2.5-stor..."


In [36]:
# seems like we can drop the property_type column but also map the listing_tags category correctly
# ^ will do this in EDA part
# Now, let's drop the columns first
try:
    df = df.drop(columns = 'property_type')
except:
    pass

assert 'property_type' not in df.columns

It seems like the following combination of columns have same information
1. psf_det & price_per_sqft
2. floor_size_det & sqft

In [37]:
df['psf_det'] = df['psf_det']\
                .str.replace('RM','')\
                .str.replace('psf','')\
                .str.replace(',','')\
                .str.strip()\
                .astype('float')

In [38]:
(df['psf_det'] - df['price_per_sqft']).sum() == 0

True

In [39]:
try:
    df = df.drop(columns = 'psf_det')
except:
    pass

assert 'psf_det' not in df.columns

In [40]:
df['floor_size_det'] = df['floor_size_det']\
                        .str.replace('sqft','')\
                        .str.replace(',','')\
                        .str.strip()\
                        .astype('float')

In [41]:
(df['floor_size_det'] - df['sqft']).sum()

-544270.0

In [42]:
df.loc[df.floor_size_det != df.sqft]

,price,listing_title,sqft,bedrooms,bathrooms,address,price_per_sqft,listing_tags,property_title_type,furnishing,tenure,floor_size_det,facilities,url,state,district
id,,,,,,,,,,,,,,,,
13,1980000.00,Sunway Rydgeway,3900.00,5.00,5.00,"Ampang Jaya, Ampang, Selangor",507.69,Bungalow House,Strata,Partially Furnished,Leasehold,3899.00,"Air-Conditioning, Balcony, Corner Unit, Park /...",https://www.propertyguru.com.my/property-listi...,Selangor,Ampang
23,3340800.00,SS 19,8740.00,5.00,5.00,"Jalan SS 19/2, Subang Jaya, Selangor",382.24,Bungalow House,Individual,NaN,Freehold,7401.00,NaN,https://www.propertyguru.com.my/property-listi...,Selangor,Subang Jaya
29,989800.00,Wira Heights,7070.00,NaN,NaN,"Jalan Wira Heights, Bandar Sungai Long, Kajang...",140.00,Residential Land,NaN,NaN,Freehold,NaN,"Jogging track, Playground, 24 hours security",https://www.propertyguru.com.my/property-listi...,Selangor,Kajang
32,5300000.00,PRIMO II @ Bukit Jelutong,10300.00,7.00,8.00,"Jalan Anjung U8/34, Bukit Jelutong, Shah Alam,...",514.56,Bungalow House,NaN,NaN,Freehold,7800.00,"Car Park, Children's Playground, Perimeter Fen...",https://www.propertyguru.com.my/property-listi...,Selangor,Shah Alam
40,2250000.00,Tijani Ukay,6000.00,6.00,6.00,"Jalan Tijani 1/B, Ampang, Selangor",375.00,Bungalow House,Strata,Partially Furnished,Leasehold,5600.00,"Air-Conditioning, Balcony, Corner Unit, Park /...",https://www.propertyguru.com.my/property-listi...,Selangor,Ampang
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2616,2200000.00,Norton Garden Bungalow @ Eco Grandeur,4000.00,5.00,5.00,"Persiaran Mokhtar Dahari, Eco Grandeur, Puncak...",550.00,Bungalow House,Strata,Unfurnished,Leasehold,3051.00,"Adult Fitness Stations, Aircon Facilities, Bas...",https://www.propertyguru.com.my/property-listi...,Selangor,Kuala Selangor
2619,1700000.00,60x100 New Launch Single & Double Storey Bungalow,6000.00,5.00,6.00,"Sungai Buloh, Selangor",283.33,Bungalow House,NaN,Unfurnished,Freehold,3000.00,NaN,https://www.propertyguru.com.my/property-listi...,Selangor,Sungai Buloh
2644,540000.00,"Prima Beruntung, Bukit Beruntung, Rawang",5000.00,5.00,4.00,"Prima Beruntung, Bukit Beruntung, Rawang, Buki...",108.00,Bungalow House,NaN,Fully Furnished,Freehold,2700.00,NaN,https://www.propertyguru.com.my/property-listi...,Selangor,Rawang


In [43]:
floor_size_diff = abs(df['floor_size_det'] - df['sqft'])
floor_size_diff

id
1      0.00
2      0.00
3      0.00
4      0.00
5      0.00
       ... 
2663   0.00
2664   0.00
2665   0.00
2666   0.00
2667   0.00
Length: 2667, dtype: float64

In [44]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2667 entries, 1 to 2667
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   price                2661 non-null   float64
 1   listing_title        2664 non-null   object 
 2   sqft                 2643 non-null   float64
 3   bedrooms             2630 non-null   float64
 4   bathrooms            2626 non-null   float64
 5   address              2661 non-null   object 
 6   price_per_sqft       2616 non-null   float64
 7   listing_tags         2666 non-null   object 
 8   property_title_type  1546 non-null   object 
 9   furnishing           1974 non-null   object 
 10  tenure               2339 non-null   object 
 11  floor_size_det       2606 non-null   float64
 12  facilities           2030 non-null   object 
 13  url                  2667 non-null   object 
 14  state                2667 non-null   object 
 15  district             2667 non-null   o

In [45]:
df['facilities'].head()

id
1    Air-Conditioning, Balcony, Water Heater, Baske...
2    Air-Conditioning, Balcony, Water Heater, Car P...
3    Air-Conditioning, Balcony, Bathtub, Maidsroom,...
4              Car Park, Perimeter Fencing, Playground
5    Air-Conditioning, Renovated, Barbeque Area, Co...
Name: facilities, dtype: object

In [46]:
# List out all the facility in the facilities columns
facilities_ls = []
for i, row in df.iterrows():
    tmp_var = str(row['facilities']).split(',')
    tmp_var = [a.strip().lower() for a in tmp_var]
    for faci in tmp_var:
        if faci not in facilities_ls:
            facilities_ls.append(faci)
            
facilities_ls

['air-conditioning',
 'balcony',
 'water heater',
 'basketball court',
 'barbeque area',
 'billiards room',
 'car park',
 'game room',
 'gymnasium room',
 'karaoke',
 'sauna',
 '24 hours security',
 'steam bath',
 'swimming pool',
 "children's playground",
 'clubhouse',
 'perimeter fencing',
 'bathtub',
 'maidsroom',
 'covered car park',
 'lounge',
 'nursery',
 'playground',
 'renovated',
 'wading pool',
 'nan',
 'jacuzzi',
 'cooker hob/hood',
 'business centre',
 'cafes',
 'squash court',
 'jogging track',
 'mini-mart',
 'tennis courts',
 'corner unit',
 'park / greenery view',
 'landscaped garden',
 'pavillion',
 'open car park',
 'basement car park',
 'drop off point',
 'lift lobby',
 'main entrance',
 'reflexology path',
 'high floor',
 'badminton hall',
 'salon',
 'original condition',
 'terrace',
 'intercom',
 'private pool',
 'audio system',
 'bed',
 'cable tv',
 'dining room furniture',
 'dishwasher',
 'dryer',
 'dvd player',
 'fridge',
 'internet connection',
 'iron / ironing 

In [47]:
df['pool'] = df['facilities'].astype('str').apply(swimming_pool)

df['fitness'] = df['facilities'].astype('str').apply(fitness)

df['balcony'] = df['facilities'].astype('str').apply(balcony)

In [48]:
facilities_ls

['air-conditioning',
 'balcony',
 'water heater',
 'basketball court',
 'barbeque area',
 'billiards room',
 'car park',
 'game room',
 'gymnasium room',
 'karaoke',
 'sauna',
 '24 hours security',
 'steam bath',
 'swimming pool',
 "children's playground",
 'clubhouse',
 'perimeter fencing',
 'bathtub',
 'maidsroom',
 'covered car park',
 'lounge',
 'nursery',
 'playground',
 'renovated',
 'wading pool',
 'nan',
 'jacuzzi',
 'cooker hob/hood',
 'business centre',
 'cafes',
 'squash court',
 'jogging track',
 'mini-mart',
 'tennis courts',
 'corner unit',
 'park / greenery view',
 'landscaped garden',
 'pavillion',
 'open car park',
 'basement car park',
 'drop off point',
 'lift lobby',
 'main entrance',
 'reflexology path',
 'high floor',
 'badminton hall',
 'salon',
 'original condition',
 'terrace',
 'intercom',
 'private pool',
 'audio system',
 'bed',
 'cable tv',
 'dining room furniture',
 'dishwasher',
 'dryer',
 'dvd player',
 'fridge',
 'internet connection',
 'iron / ironing 

In [49]:
try:
    df = df.drop(columns = 'facilities')
except:
    pass

assert 'facilities' not in df.columns

## Impute missing values

### Drop observations & features that contains huge amount of missing values

In the 'RawDataPipeline', we have filtered out the columns with more than 50% proportion of missing values.

In [50]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2667 entries, 1 to 2667
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   price                2661 non-null   float64
 1   listing_title        2664 non-null   object 
 2   sqft                 2643 non-null   float64
 3   bedrooms             2630 non-null   float64
 4   bathrooms            2626 non-null   float64
 5   address              2661 non-null   object 
 6   price_per_sqft       2616 non-null   float64
 7   listing_tags         2666 non-null   object 
 8   property_title_type  1546 non-null   object 
 9   furnishing           1974 non-null   object 
 10  tenure               2339 non-null   object 
 11  floor_size_det       2606 non-null   float64
 12  url                  2667 non-null   object 
 13  state                2667 non-null   object 
 14  district             2667 non-null   object 
 15  pool                 2667 non-null   b

We can see what is the proportion of missing values for each columns

In [51]:
missing_prop = (df.isna().sum()) / df.shape[0] * 100
missing_prop.sort_values(ascending = False)

property_title_type   42.03
furnishing            25.98
tenure                12.30
floor_size_det         2.29
price_per_sqft         1.91
bathrooms              1.54
bedrooms               1.39
sqft                   0.90
price                  0.22
address                0.22
listing_title          0.11
listing_tags           0.04
url                    0.00
state                  0.00
district               0.00
pool                   0.00
fitness                0.00
balcony                0.00
dtype: float64

In [52]:
# property title type\
property_title_type_na = (df['property_title_type'].isna().sum() / df.shape[0]) * 100
print(f'Missing Values: {round(property_title_type_na,2)}%')

Missing Values: 42.03%


In [53]:
# since the missing values proportion is quite big also, we can drop the column
try:
    df = df.drop(columns = ['property_title_type'])
except:
    pass

assert 'property_title_type' not in df.columns

In [54]:
missing_prop = (df.isna().sum()) / df.shape[0] * 100
missing_prop.sort_values(ascending = False)

furnishing       25.98
tenure           12.30
floor_size_det    2.29
price_per_sqft    1.91
bathrooms         1.54
bedrooms          1.39
sqft              0.90
price             0.22
address           0.22
listing_title     0.11
listing_tags      0.04
url               0.00
state             0.00
district          0.00
pool              0.00
fitness           0.00
balcony           0.00
dtype: float64

In [55]:
df.loc[df.floor_size_det.isna()][['floor_size_det','sqft']]

,floor_size_det,sqft
id,,
29,NaN,7070.00
66,NaN,8000.00
74,NaN,8342.00
88,NaN,5.00
99,NaN,NaN
...,...,...
2351,NaN,5815.00
2430,NaN,5112.00
2476,NaN,6006.00


In [56]:
df.loc[(df.floor_size_det.isna()) & (~df.sqft.isna())][['floor_size_det','sqft']]

,floor_size_det,sqft
id,,
29,NaN,7070.00
66,NaN,8000.00
74,NaN,8342.00
88,NaN,5.00
150,NaN,14691.00
268,NaN,10325.00
290,NaN,9182.00
349,NaN,7233.00
364,NaN,3.00


In [57]:
df.loc[(~df.floor_size_det.isna()) & (df.sqft.isna())][['floor_size_det','sqft']]

,floor_size_det,sqft
id,,


We can see that the missing values in floor_size_det is more than sqft.

In [58]:
# Special indicator for EDA use
df.loc[(df.floor_size_det.isna()) & (~df.sqft.isna()),'floor_size_det'] = -99999

In [59]:
target_cols = missing_prop[(missing_prop <= 2)].index.tolist()
target_cols

['price',
 'listing_title',
 'sqft',
 'bedrooms',
 'bathrooms',
 'address',
 'price_per_sqft',
 'listing_tags',
 'url',
 'state',
 'district',
 'pool',
 'fitness',
 'balcony']

Since the above features contain less than 2% of missing values, we can delete the observations.

In [60]:
df = df.dropna(axis = 0, how = 'any', subset = target_cols)

In [61]:
missing_prop = (df.isna().sum()) / df.shape[0] * 100
missing_prop.sort_values(ascending = False)

furnishing       24.44
tenure           12.20
fitness           0.00
pool              0.00
district          0.00
state             0.00
url               0.00
floor_size_det    0.00
price             0.00
listing_title     0.00
listing_tags      0.00
price_per_sqft    0.00
address           0.00
bathrooms         0.00
bedrooms          0.00
sqft              0.00
balcony           0.00
dtype: float64

In [62]:
print(f'Number of rows removed: {df_ori.shape[0] - df.shape[0]}')

Number of rows removed: 85


Since only 140 rows are removed, it won't has huge impact on our analysis further on.

### Missing Values imputation

In [63]:
df['furnishing'].value_counts(normalize = True)
# Impute using mode 
df['furnishing'].fillna(df['furnishing'].mode().values[0], inplace = True)

In [64]:
df['tenure'].value_counts(normalize = True)

Freehold              0.67
Leasehold             0.30
Malay Reserved Land   0.01
99-year Leasehold     0.01
Bumi Lot              0.01
Name: tenure, dtype: float64

In [65]:
# Impute using mode 
df['tenure'].fillna(df['tenure'].mode().values[0], inplace = True)

In [66]:
df['tenure'].value_counts(normalize = True)

Freehold              0.71
Leasehold             0.26
Malay Reserved Land   0.01
99-year Leasehold     0.01
Bumi Lot              0.01
Name: tenure, dtype: float64

In [67]:
assert df['tenure'].isna().sum() == 0

In [68]:
df.isna().sum()

price             0
listing_title     0
sqft              0
bedrooms          0
bathrooms         0
address           0
price_per_sqft    0
listing_tags      0
furnishing        0
tenure            0
floor_size_det    0
url               0
state             0
district          0
pool              0
fitness           0
balcony           0
dtype: int64

In [69]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2582 entries, 1 to 2667
Data columns (total 17 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   price           2582 non-null   float64
 1   listing_title   2582 non-null   object 
 2   sqft            2582 non-null   float64
 3   bedrooms        2582 non-null   float64
 4   bathrooms       2582 non-null   float64
 5   address         2582 non-null   object 
 6   price_per_sqft  2582 non-null   float64
 7   listing_tags    2582 non-null   object 
 8   furnishing      2582 non-null   object 
 9   tenure          2582 non-null   object 
 10  floor_size_det  2582 non-null   float64
 11  url             2582 non-null   object 
 12  state           2582 non-null   object 
 13  district        2582 non-null   object 
 14  pool            2582 non-null   bool   
 15  fitness         2582 non-null   bool   
 16  balcony         2582 non-null   bool   
dtypes: bool(3), float64(6), object(8)

In [70]:
df.reset_index().to_csv('data/Processed_data_for_EDA.csv', index = False)